In [5]:
%pip install pandas nltk scikit-learn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 718.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.0 MB/s eta 0:00:00
Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.7/281.7 kB 4.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 3.7 MB/s eta 0:00:00
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://

NLP II - Lecture 4: Logistic Regression

In [4]:
import pandas as pd
data = pd.DataFrame({
    'text': ['meow meow yawn','nyan nyan ow','yawn woof woof', 'hong ow', 'yawn ow', 'hong ow', 'woof hong', 'nyan hong'],
    'label': ['cat','cat','dog','dog','camel','camel','dog','dog']
})
data

,text,label
0,meow meow yawn,cat
1,nyan nyan ow,cat
2,yawn woof woof,dog
3,hong ow,dog
4,yawn ow,camel
5,hong ow,camel
6,woof hong,dog
7,nyan hong,dog


In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/Fai/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from nltk import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# load data
data = pd.DataFrame({
    'text': ['meow meow yawn','nyan nyan ow','yawn woof woof', 'hong ow', 'yawn ow', 'hong ow', 'woof hong', 'nyan hong'] * 50,
    'label': ['cat','cat','dog','dog','camel','camel','dog','dog'] * 50
})

# prepare data - random_state create same shuffle result 
train, dev_test = train_test_split(data, test_size=0.8, random_state=140)
dev, test = train_test_split(dev_test, test_size=0.5, random_state=140)

# feature engineering
def featureize(text):
    dict_feature = {}
    dict_feature['ความยาวของข้อความ'] = len(text)
    for token in word_tokenize(text):
        if token not in dict_feature:
            dict_feature[token] = 0
        dict_feature[token] += 1
    return dict_feature
    # return {x:1 for x in word_tokenize(text)}
vectorizer = DictVectorizer(sparse=True)
train_features = train['text'].apply(featureize)
feature_vectors = vectorizer.fit_transform(train_features)

# train model
lr_text_classifier = LogisticRegression()
lr_text_classifier.fit(feature_vectors, train['label'])

# evaluate model
dev_feature = dev['text'].apply(featureize)
dev_feature_vectors = vectorizer.transform(dev_feature)


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
weights = pd.DataFrame(lr_text_classifier.coef_, columns=vectorizer.feature_names_,index=lr_text_classifier.classes_)

In [17]:
weights

,hong,meow,nyan,ow,woof,yawn,ความยาวของข้อความ
camel,-0.622195,0.000272,-0.408771,0.481495,-0.099255,0.640291,-1.010996
cat,-0.342306,0.625321,0.727776,0.429591,-0.915392,-0.169703,0.920177
dog,0.964501,-0.625594,-0.319005,-0.911086,1.014647,-0.470588,0.090819


In [18]:
weights.transpose()

,camel,cat,dog
hong,-0.622195,-0.342306,0.964501
meow,0.000272,0.625321,-0.625594
nyan,-0.408771,0.727776,-0.319005
ow,0.481495,0.429591,-0.911086
woof,-0.099255,-0.915392,1.014647
yawn,0.640291,-0.169703,-0.470588
ความยาวของข้อความ,-1.010996,0.920177,0.090819


In [21]:
weights.transpose().sort_values('dog', ascending=False)

,camel,cat,dog
woof,-0.099255,-0.915392,1.014647
hong,-0.622195,-0.342306,0.964501
ความยาวของข้อความ,-1.010996,0.920177,0.090819
nyan,-0.408771,0.727776,-0.319005
yawn,0.640291,-0.169703,-0.470588
meow,0.000272,0.625321,-0.625594
ow,0.481495,0.429591,-0.911086


In [22]:
weights.transpose().loc['nyan']

camel   -0.408771
cat      0.727776
dog     -0.319005
Name: nyan, dtype: float64

In [ ]:
weights.to_excel('weights.xlsx')

In [23]:
# bias
pd.DataFrame(lr_text_classifier.intercept_, index=lr_text_classifier.classes_)

,0
camel,9.078864
cat,-10.031241
dog,0.952376
